INGESTA DE DATOS - EXTRACCIÓN

Importamos las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import requests as rq
import io

Realizamos la extraccion de los datos utilizando la libreria request

In [ ]:
req_amazon = rq.get('https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/amazon_prime_titles.csv').content

req_disney = rq.get('https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/disney_plus_titles.csv').content

req_hulu = rq.get('https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/hulu_titles.csv').content

req_netflix = rq.get('https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/netflix_titles.json').content


Convertimos las request obtenidas en "Dataframes" de Pandas

In [ ]:
df_amazon = pd.read_csv(io.StringIO(req_amazon.decode('utf-8')))

df_disney = pd.read_csv(io.StringIO(req_disney.decode('utf-8')))

df_hulu = pd.read_csv(io.StringIO(req_hulu.decode('utf-8')))

df_netflix = pd.read_json(io.StringIO(req_netflix.decode('utf-8')))

Verficamos la informacion de cada dataframe

In [ ]:
df_amazon.head(2)

In [ ]:
df_disney.head(2)

In [ ]:
df_hulu.head(2)

In [ ]:
df_netflix.head(19)

NORMALIZACIÓN DE DATOS

Modificamos ID's para poder diferenciarlos por plataforma

In [ ]:
df_amazon.show_id = df_amazon.show_id + '_amz'
df_disney.show_id = df_disney.show_id + '_dis'
df_hulu.show_id = df_hulu.show_id + '_hl'
df_netflix.show_id = df_netflix.show_id + '_ntf'

Concatenamos (unimos) los 4 dataframes en un único dataframe para poder realizar las consultas

In [ ]:
df_plataformas = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], axis=0)

In [ ]:
df_plataformas.head(2)

Dividimos los datos de la columna "duration" en 2 columnas: "duration" (con el valor numérico de la duración) y "min/season" (con el tipo de valor ya sea minutos o season)

In [ ]:
temp = df_plataformas["duration"].str.split(" ", n = 1, expand = True) 
  
df_plataformas.drop(columns =["duration"], inplace = True)

df_plataformas["duration"]= temp[0]

df_plataformas["min_season"]= temp[1]
  
df_plataformas

Convertimos el tipo de datos de la columna "duration" a Float

In [ ]:
df_plataformas['duration'] = df_plataformas['duration'].astype(float)

Elimino las columnas que considero innecesarias para las consultas del proyecto (solo las elimino del Dataframe df_plataformas). Si se presenta la situacion de que la compañia/empresa necesite algunos de estos datos en el futuro puedo obtenerlos de los 4 dataframes creados al inicio del script.

In [ ]:
df_plataformas.drop(['date_added','rating', 'description'], axis = 'columns', inplace=True)

Verificamos la existencia de valores nulos por columna

In [ ]:
print(df_plataformas['show_id'].isna().sum())
print(df_plataformas['type'].isna().sum())
print(df_plataformas['title'].isna().sum())
print(df_plataformas['director'].isna().sum())
print(df_plataformas['cast'].isna().sum())
print(df_plataformas['country'].isna().sum())
print(df_plataformas['release_year'].isna().sum())
print(df_plataformas['listed_in'].isna().sum())
print(df_plataformas['duration'].isna().sum())
print(df_plataformas['min_season'].isna().sum())

Modificamos los valores nulos de acuerdo a la siguiente condicion:
- Si la columna es tipo string (object en Pandas), los NaN se reemplazan por 'Sin dato'.
- Si la columna es de tipo float, los NaN reemplazan por la Mediana de la columna. La elección de la mediana (85) se debe a que la mayoria de las peliculas duran más de 1 hora y el valor de la media es 67 (aproximadamente 1 hora).

In [ ]:
df_plataformas.fillna({'director': 'Sin dato', 'cast': 'Sin dato', 'country': 'Sin dato', 'duration': df_plataformas['duration'].median() , 'min_season': 'Sin dato'}, inplace=True)

In [ ]:
df_plataformas['min_season'].replace({"Season":'season',"Seasons":'season'}, inplace=True)

In [26]:
df_plataformas

,show_id,type,title,director,cast,country,release_year,listed_in,duration,min_season
0,s1_amz,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2014,"Comedy, Drama",113.0,min
1,s2_amz,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2018,"Drama, International",110.0,min
2,s3_amz,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2017,"Action, Drama, Suspense",74.0,min
3,s4_amz,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2014,Documentary,69.0,min
4,s5_amz,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,1989,"Drama, Fantasy",45.0,min
...,...,...,...,...,...,...,...,...,...,...
8802,s8803_ntf,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2007,"Cult Movies, Dramas, Thrillers",158.0,min
8803,s8804_ntf,TV Show,Zombie Dumb,Sin dato,Sin dato,Sin dato,2018,"Kids' TV, Korean TV Shows, TV Comedies",2.0,season
8804,s8805_ntf,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2009,"Comedies, Horror Movies",88.0,min
8805,s8806_ntf,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2006,"Children & Family Movies, Comedies",88.0,min
